# Chimeras - A mixture of red pandas and lions

#### By Eneritz Domínguez and Ander Salaberria

During this project, we have had in mind two different approaches to achieve our goal. We'll show our built DiscoGAN after explaining how we got the images, and then we'll show a fully connected neural network that learns to differenciate between red pandas, lions and the background of the image (image segmentation).

All the needed images and models are saved in GitHub. It is preferable to download it, and use the notebook that is already in that repository. Check it out here: http://github.com/eneritznely/Chimeras.git

In [2]:
# imports needed in part 1
import requests
from io import BytesIO
from PIL import Image

import os
import sys

# imports needed in part 2
import tensorflow as tf
import matplotlib.pyplot as plt

# imports needed in part 3
import numpy as np
import scipy.misc as misc

### 1. Getting Images

These two blocks and a .txt file are enough to get the images we need. These .txt contains a list of URL where images of one class are contained (lion, for instance). Those lists were extracted from www.image-net.org.

In [ ]:
# Get image from URL
def getImage(url, imgID, imgTarget):

    try:
        response = requests.get(url)
    except:
        return "fail request - " + str(imgID)
    try:
        img = Image.open(BytesIO(response.content))
    except:
        return "no image - " + str(imgID)
    
    file_name = str(imgTarget) + "_" + str(imgID) + ".jpeg"
    
    try:
        img.save(file_name);
    except:
        return "fail save - " + str(imgID)
    
    return ""

In [ ]:
target = "Lion"
images = open("Datasets/Lion_URL.txt", "r")
if not os.path.exists(target):
    os.mkdir(target)
os.chdir(target)
i = 0
for line in images:
    i = i + 1
    response = getImage(line, i, target)
    if not response == "":
        print(response)
os.chdir("../")

Our model only deals with 64x64 images, so we need to resize them. It is necessary to change the directory of variable "path" in order to specify where is our dataset.

In [ ]:
# resizes all the images in an directory "path" into 64x64 size 
def resize(path):
    dirs = os.listdir(path)
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((64,64), Image.ANTIALIAS)
            imResize.save(f + '.jpeg', 'JPEG', quality=90)

In [ ]:
path = "Lion/"
resize(path)

### 2. DISCOvery Cross-Domain Relations with GANs (DiscoGAN)

This implementation is in Tensorflow, so, firstly, we are going to define our hyperparameters and reset the tensorflow graph.

In [3]:
# reset tensorflow graph
tf.reset_default_graph()

# Hyperparameters
initializer = tf.truncated_normal_initializer(stddev=0.02) # define weights with mean 0 and standard deviaton of 0.02
learning_rate = 0.0005
batch_size = 250
epoch = 100000

#### Preprocessing input images

After this, we are going to detect and read both red panda and lion images. We'll do all this with tensorflow's tensors. So, these cells are going to define the first part of the graph where all images are read and preprocessed in an appropiate way.

In [4]:
# Read images 
red_pandas_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("Red_panda_64x64/*.jpeg"), capacity=200)
lions_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("Lion_64x64/*.jpeg"), capacity=200)

image_reader = tf.WholeFileReader()
_, red_pandas_file = image_reader.read(red_pandas_filename_queue)
_, lions_file = image_reader.read(lions_filename_queue)

In [5]:
# Decode images so each image is composed by width x height x channel integers/pixel-values
red_pandas_image = tf.image.decode_jpeg(red_pandas_file)
lions_image = tf.image.decode_jpeg(lions_file)

In [6]:
# we will normalize the pixel values, so black stays as RGB = [0.0, 0.0, 0.0] and white becomes RGB = [1.0, 1.0, 1.0]
red_pandas_image = tf.cast(tf.reshape(red_pandas_image,shape=[64,64,3]),dtype=tf.float32)/255.0
lions_image = tf.cast(tf.reshape(lions_image,shape=[64,64,3]),dtype=tf.float32)/255.0

With the shuffle_batch function, we are going to create a batch which is different on each iteration, beacuse they are randomly shuffled. 

In [7]:
num_preprocess_threads = 1 #number of threads used to shuffle the batches
min_queue_examples = 250 # min number of elements admitted in queue

batch_red_pandas = tf.train.shuffle_batch([red_pandas_image], # images which will be contained in the batch
                                     batch_size=batch_size,   # number of images in a batch
                                     num_threads=num_preprocess_threads, # number of threads that will execute this
                                     capacity=min_queue_examples + 3 * batch_size, # max number of elements in queue
                                     min_after_dequeue=min_queue_examples) # min elements in queue after a dequeue

batch_lions = tf.train.shuffle_batch([lions_image],
                                    batch_size=batch_size,
                                    num_threads=num_preprocess_threads,
                                    capacity=min_queue_examples + 3 * batch_size,
                                    min_after_dequeue=min_queue_examples)

#### Defining neural networks - Generators & Discriminators

Once we have all the preprocessing defined, let's define our discriminators and generators of the DiscoGAN. 

On the one hand, we've got two discriminators, which are CNNs with 5 convolutional layers each, followed by 2 fully connected layers. We use the ReLU activation function (except in the last layer, where we use the sigmoid function), kernel size of 4 and stride of 2. The weights are initialized as defined in the hyperparameters and a normalization function is used instead of biases.

In [8]:
# with this function we define a discriminator, using the following parameters:
#   - tensor: the input tensor of the image
#   - name: the name we'll use to specify the discriminator we are creating/reusing
#   - reuse: if the discriminator isn't already created (FALSE) or if we'll be using a existing discriminator (TRUE)

def discriminator(tensor,name,reuse=False):
    
    with tf.variable_scope(name):
        conv1 = tf.contrib.layers.conv2d(inputs=tensor, num_outputs=32, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,weights_initializer=initializer,scope="d_conv1") # 32 x 32 x 32
        conv2 = tf.contrib.layers.conv2d(inputs=conv1, num_outputs=64, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv2")                                  # 16 x 16 x 64
        conv3 = tf.contrib.layers.conv2d(inputs=conv2, num_outputs=128, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv3")                                  # 8 x 8 x 128 
        conv4 = tf.contrib.layers.conv2d(inputs=conv3, num_outputs=256, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv4")                                  # 4 x 4 x 256
        conv5 = tf.contrib.layers.conv2d(inputs=conv4, num_outputs=512, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv5")                                  # 2 x 2 x 512
        fc1 = tf.reshape(conv5, shape=[batch_size, 2*2*512])
        fc1 = tf.contrib.layers.fully_connected(inputs=fc1, num_outputs=512,reuse=reuse, activation_fn=tf.nn.relu, \
                                                normalizer_fn=tf.contrib.layers.batch_norm, \
                                                weights_initializer=initializer,scope="d_fc1")
        fc2 = tf.contrib.layers.fully_connected(inputs=fc1, num_outputs=1, reuse=reuse, activation_fn=tf.nn.sigmoid,\
                                                weights_initializer=initializer,scope="d_fc2")

        return fc2

On the other hand, we've got our four generators. These have an encoder-decoder architecture meaning that both the input and output are images. We use 4 encoders (convolution + activation function) to hopefully get a higher level representation of the image. Another 4 decode layers do the opposite (deconvolution + activation function) and reverse the action of the encoder layers.

In these case, all layers use our beloved ReLU activation function and kernel size of 4. The stride is 2 for encoder layers and 1 for decoder layers. After each deconvolution, a reshape is done to increase the dimensions (height and width) of the image, reducing the channel number.

In [9]:
# with this function we define a generator, using the following parameters:
#   - image: the input tensor of the image
#   - name: the name we'll use to specify the generator we are creating/reusing
#   - reuse: if the generator isn't already created (FALSE) or if we'll be using a existing discriminator (TRUE)

def generator(image,name,reuse=False):

    with tf.variable_scope(name,reuse=reuse):
        conv1 = tf.contrib.layers.conv2d(inputs=image, num_outputs=32, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,weights_initializer=initializer,scope="d_conv1") # 32 x 32 x 32
        conv2 = tf.contrib.layers.conv2d(inputs=conv1, num_outputs=64, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv2")                                  # 16 x 16 x 64
        conv3 = tf.contrib.layers.conv2d(inputs=conv2, num_outputs=128, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv3")                                  # 8 x 8 x 128 
        conv4 = tf.contrib.layers.conv2d(inputs=conv3, num_outputs=256, kernel_size=4, stride=2, padding="SAME", \
                                        reuse=reuse, activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm,\
                                        weights_initializer=initializer,scope="d_conv4")                                  # 4 x 4 x 256
        
        conv_trans1 = tf.contrib.layers.conv2d(conv4, num_outputs=4*128, kernel_size=4, stride=1, padding="SAME",    \
                                        activation_fn=tf.nn.relu, normalizer_fn=tf.contrib.layers.batch_norm, \
                                        weights_initializer=initializer,scope="g_conv1")                            
        conv_trans1 = tf.reshape(conv_trans1, shape=[batch_size,8,8,128])                                                 # 8 x 8 x 128

        conv_trans2 = tf.contrib.layers.conv2d(conv_trans1, num_outputs=4*64, kernel_size=4, stride=1, padding="SAME", \
                                        activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm, \
                                        weights_initializer=initializer,scope="g_conv2")
        conv_trans2 = tf.reshape(conv_trans2, shape=[batch_size,16,16,64])                                                # 16 x 16 x 128

        conv_trans3 = tf.contrib.layers.conv2d(conv_trans2, num_outputs=4*32, kernel_size=4, stride=1, padding="SAME",    \
                                        activation_fn=tf.nn.relu, normalizer_fn=tf.contrib.layers.batch_norm, \
                                        weights_initializer=initializer,scope="g_conv3")
        conv_trans3 = tf.reshape(conv_trans3, shape=[batch_size,32,32,32])                                                # 32 x 32 x 32

        conv_trans4 = tf.contrib.layers.conv2d(conv_trans3, num_outputs=4*16, kernel_size=4, stride=1, padding="SAME", \
                                        activation_fn=tf.nn.relu,normalizer_fn=tf.contrib.layers.batch_norm, \
                                        weights_initializer=initializer,scope="g_conv4")
        conv_trans4 = tf.reshape(conv_trans4, shape=[batch_size,64,64,16])                                                # 64 x 64 x 16

        recon = tf.contrib.layers.conv2d(conv_trans4, num_outputs=3, kernel_size=4, stride=1, padding="SAME", \
                                        activation_fn=tf.nn.relu,scope="g_conv5")                                         # 64 x 64 x 3


        return recon


Now we are going to build our DNN architecture (DiscoGAN), describing our generators, discriminators and connections between then.

In [10]:
#########################
# DiscoGAN Architecture #
#########################

gen_l_fake = generator(batch_red_pandas, "generator_rp_to_l") # picks a real red panda image and creates a fake lion 
gen_rp_fake = generator(batch_lions, "generator_l_to_rp") # picks a real lion image and creates a fake red panda 

# these two generators share weights and characteristics with the first two generators
recon_rp = generator(gen_l_fake, "generator_l_to_rp", reuse=True) # takes a fake chimera and tries to get the real panda
recon_l = generator(gen_rp_fake, "generator_rp_to_l", reuse=True) # same but the chimera was based on a lion

# Discriminators
disc_rp_fake = discriminator(gen_rp_fake, "discriminator_rp") # defining discriminator with red panda fake inputs
disc_l_fake = discriminator(gen_l_fake, "discriminator_l") # defining discriminator with lion fake inputs

# these two discriminators are the same as the first two (shares weights and characteristics), but we add them
# another possible input, real images
disc_rp_real = discriminator(batch_red_pandas, "discriminator_rp", reuse=True) 
disc_l_real = discriminator(batch_lions, "discriminator_l", reuse=True)

#### Computing loss and gradients

Loss function is calculated differently for generators and discriminators. Discriminator's loss is only determined by computing sigmoid cross entropy with logits (the inverse function of the sigmoid). Generators loss is calculated using the sigmoid cross entropy, as well, of the predictions that the discriminator made from the fake images that this generator has created; and adding the MSE of the difference between the input-real image and the reconstructed one after passing through the two different generators.

In [ ]:
##################
# Loss Functions #
##################

# MSE loss calculation between the input-real image and the reconstructed one after passing through two generators  
const_loss_rp = tf.reduce_sum(tf.losses.mean_squared_error(batch_red_pandas, recon_rp))
const_loss_l = tf.reduce_sum(tf.losses.mean_squared_error(batch_lions, recon_l))

gen_rp_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_rp_fake, 
                                                                labels=tf.ones_like(disc_rp_fake)))
gen_l_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_l_fake, 
                                                                labels=tf.ones_like(disc_l_fake)))

disc_rp_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_rp_real, 
                                                                labels=tf.ones_like(disc_rp_real))) \
                            + tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_rp_fake, 
                                                                labels=tf.zeros_like(disc_rp_fake))
disc_l_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_l_real, 
                                                                labels=tf.ones_like(disc_l_real))) \
                            + tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_l_fake, 
                                                                labels=tf.zeros_like(disc_l_fake))

# a value lambda is multiplied with const_loss_rp/l to give them more relevance, to get better reconstructions faster
gen_loss = tf.constant(5.0)*tf.constant(250.0)*(const_loss_rp + const_loss_l) + gen_rp_loss + gen_l_loss
disc_loss = disc_rp_loss + disc_l_loss

Finally, after each epoch, we need to compute the gradients of both discriminators and generators. We'll compute gradients for the trainable variables taking into account the loss of both types of deep neural networks, and apply gradients to those variables.  

In [ ]:
#############################
# Compute & Apply gradients #
#############################

# Getting training variables (weights) of both generators and discriminators
gen_rp_to_l_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="generator_rp_to_l")
gen_l_to_rp_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="generator_l_to_rp")
dis_rp_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator_rp") 
dis_l_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="discriminator_l") 

# We will use the ADAM optimizer algorithm
d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# The gradients will be computed in two turns, first those of the discriminator and next of the generators
d_grads = d_optimizer.compute_gradients(disc_loss,dis_rp_variables + dis_l_variables) 
g_grads = g_optimizer.compute_gradients(gen_loss,gen_rp_to_l_variables + gen_l_to_rp_variables)

# Finally, we will update the gradients
update_D = d_optimizer.apply_gradients(d_grads)
update_G = g_optimizer.apply_gradients(g_grads)

#### Let's Train!

We've got all we need! Input-data, a good looking architecture of neural networks, loss functions and a gradient computing algorithm. Now it's time to have patience, because around 90 epochs lasts an hour and the improvement is quite slow.

In [ ]:
#################
# Training Time #
#################

init = tf.global_variables_initializer()
init2 = tf.local_variables_initializer()

# we define a saver in order to save values of the trainable variables every 20 epochs
saver = tf.train.Saver()

with tf.Session() as sess:
    # we initialize local and global variables
    sess.run(init)
    sess.run(init2)

    # if a session is stored, we'll load it
    try:
        saver.restore(sess=sess, save_path="./DiscoGAN_logs/model.ckpt")
        print("\n--------model restored--------\n")
    except:
        print("\n--------model Not restored--------\n")
        pass

    # creates a coordinator to coordinate the termination of a set of threads
    coord = tf.train.Coordinator()
    # and threads start running 
    threads = tf.train.start_queue_runners(coord=coord)
    
    # we start itertating among epochs
    for i in range(epoch):
        
        # in each epoch, trainable variables of generators are updated three times more than the discriminator's variables
        # it's a way to impulse the generator to achieve better images and stay in tune with the discriminator's quality
        for j in range(2):
                _ = sess.run([update_G])
        
        # finally, we update the whole network and get 6 images, two of them real input images and the other 4 fakes
        _, _, g_loss, d_loss, reconed_l, reconed_rp, fake_rp, fake_l,rp_image, l_image  = sess.run([update_G,update_D, 
                                                                                                gen_loss, disc_loss,
                                                                                                recon_l,recon_rp,
                                                                                                gen_rp_fake,gen_l_fake,
                                                                                                batch_red_pandas,batch_lions])

        # every 10 epochs the loss of the generators and discriminators are shown, to see how well they are doing
        if i % 10 == 0:
            print("{}th iter gen loss:{} disc loss:{}".format(i,g_loss/batch_size, d_loss/batch_size))
            
        # every 20 epochs the model is saved next to the 6 images: 2 inputs, 2 generated chimeras and 2 reconstructions
        if i % 20 == 0:
            saver.save(sess, './DiscoGAN_logs/model.ckpt')
            plt.imsave("./DiscoGAN_results/{}th_recon_l.png".format(i),reconed_l[0])
            plt.imsave("./DiscoGAN_results/{}th_recon_rp.png".format(i),reconed_rp[0])
            plt.imsave("./DiscoGAN_results/{}th_origin_rp.png".format(i),rp_image[0])
            plt.imsave("./DiscoGAN_results/{}th_origin_l.png".format(i),l_image[0])
            plt.imsave("./DiscoGAN_results/{}th_gen_rp.png".format(i),fake_rp[0])
            plt.imsave("./DiscoGAN_results/{}th_gen_l.png".format(i),fake_l[0])
    

    # once it is finished, stop threads
    coord.request_stop()
    coord.join(threads)


### 3. The idea of chimeras based on image segmentation

This was our original idea, but it was too complex to make it real. As mentioned in our report, it consisted of 3 different parts, but we only implemented the first before discovering DiscoGAN (no pun intended).

The first part is about image segmentation. Our idea was to, given a picture, detect a red panda or lion in the image and detect where exactly was in the image. So we need a neural network capable to learn and return a segmentation of a given image between red panda, lion and background.

Fully Convolutional Networks (FCN) are mostly used in this area, and digging a little bit in github, we found a great FCN focused on detecting liquid, glass, mirrors and so on: https://github.com/sagieppel/Fully-convolutional-neural-network-FCN-for-semantic-segmentation-Tensorflow-implementation 

So, we thought of using that code in order to train this FCN to detect lions and red pandas.

#### Training FCN for image segmentation

Before doing anything, download our mentioned repository from GitHub and extract its content in the same directory as this notebook. The FCN is already trained, so there is no need to execute the training part.

In [ ]:
# Imports to execute training and evaluation of the FCN
sys.path.insert(0, 'FCN_code')

import Data_Reader
import BuildNetVgg16
import CheckVGG16Model
import OverrlayLabelOnImage as Overlay
import TensorflowUtils

In [ ]:
########################
# Input/Output Folders #
########################

Train_Image_Dir = "Animals_FCN/"  # Images and labels for training
Train_Label_Dir = "Animals_FCN_segmented"  # Annotation in png format for training images
logs_dir = "FCN_logs/"  # Path to logs directory where trained model and information will be stored
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)
model_path = "vgg16/vgg16.npy"  # Path to pretrained vgg16 model for encoder
CheckVGG16Model.CheckVGG16(model_path)  # Check if pretrained vgg16 model is available; if not, try to download 

In [ ]:
####################
# Hyper-parameters #
####################

learning_rate = 1e-5  # Learning rate for Adam Optimizer
TrainLossTxtFile = logs_dir + "TrainLoss.txt"  # Where train losses will be writen
Batch_Size = 1  # Number of files per training iteration
Weight_Loss_Rate = 5e-4  # Weight for the weight decay loss function
MAX_ITERATION = 100000  # Max number of training iteration
NUM_CLASSES = 3  # 0: Background - 1: Red Pandas - 2: Lions

In [ ]:
#######################
# Gradient Definition #
#######################

def train(loss_val, var_list):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads = optimizer.compute_gradients(loss_val, var_list=var_list)
    return optimizer.apply_gradients(grads)

In [ ]:
#####################
# Training Function #
#####################

def trainingFCN(argv=None):
    
    tf.reset_default_graph()
    keep_prob = tf.placeholder(tf.float32, name="keep_probabilty")  # Dropout probability

    # Placeholders for input image and labels
    image = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input_image")  # Input image batch first dimension image number second dimension width third dimension height 4 dimension RGB
    GTLabel = tf.placeholder(tf.int32, shape=[None, None, None, 1], name="GTLabel")  # Ground truth labels for training

    # Build FCN Net
    Net = BuildNetVgg16.BUILD_NET_VGG16(vgg16_npy_path=model_path)  # Create class for the network
    Net.build(image, NUM_CLASSES, keep_prob)  # Create the net and load intial weights

    # Get loss functions for neural net work  one loss function for each set of label
    Loss = tf.reduce_mean((tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.squeeze(GTLabel, squeeze_dims=[3]), logits=Net.Prob, name="Loss")))  # Define loss function for training

    trainable_var = tf.trainable_variables()  # Collect all trainable variables for the net
    train_op = train(Loss, trainable_var)  # Create Train Operation for the net

    # Create reader for data set
    TrainReader = Data_Reader.Data_Reader(Train_Image_Dir,  GTLabelDir=Train_Label_Dir, BatchSize=Batch_Size)
    
    
    sess = tf.Session()  # Start Tensorflow session

    # load trained model if exist
    print("Setting up Saver...")
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer()) #Initialize variables
    ckpt = tf.train.get_checkpoint_state(logs_dir)
    if ckpt and ckpt.model_checkpoint_path: # if train model exist restore it
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")

    # Create files in order to save loss
    f = open(TrainLossTxtFile, "w")
    f.write("Iteration\tloss\t Learning Rate="+str(learning_rate))
    f.close()

    # Start Training loop: Main Training
    for itr in range(MAX_ITERATION):
        Images,  GTLabels = TrainReader.ReadAndAugmentNextBatch()  # Load augmeted images and ground true labels for training
        feed_dict = {image: Images, GTLabel: GTLabels, keep_prob: 0.5}
        sess.run(train_op, feed_dict=feed_dict)  # Train one cycle

        # Save trained model
        if itr % 500 == 0 and itr>0:
            print("Saving Model to file in "+logs_dir)
            saver.save(sess, logs_dir + "model.ckpt", itr)  # Save model

        # Write and display train loss
        if itr % 10 == 0:
            # Calculate train loss
            feed_dict = {image: Images, GTLabel: GTLabels, keep_prob: 1}
            TLoss = sess.run(Loss, feed_dict=feed_dict)
            print("Step "+str(itr)+" Train Loss="+str(TLoss))
            # Write train loss to file
            with open(TrainLossTxtFile, "a") as f:
                f.write("\n"+str(itr)+"\t"+str(TLoss))
                f.close()

In [ ]:
##############
# Run Script #
##############

trainingFCN()
print("Finished")

#### Evaluating FCN for image segmentation

In this part, you are going to test this FCN with some test images. Those images are already in Test directory and the output segmentations will be saved automatically. 

In [ ]:
########################
# Input/Output Folders #
########################

logs_dir= "FCN_logs" # path to logs directory where trained model and information will be stored
Image_Dir="Lion_64x64/" # CHANGE IT if you want, it's the test image folder
Pred_Dir="FCN_results/" # Directory where the output prediction will be written
NameEnd="" # Add this string to the ending of the file name optional
model_path="vgg16/vgg16.npy" # Path to pretrained vgg16 model for encoder
CheckVGG16Model.CheckVGG16(model_path)# Check if pretrained vgg16 model avialable and if not try to download it

In [ ]:
###################
# Hyperparameters #
###################

w=0.6 # weight of overlay on image
NUM_CLASSES = 3 # Number of classes

In [ ]:
def evaluatingFCN(argv=None):
    
    # Placeholders for input image and labels
    keep_prob = tf.placeholder(tf.float32, name="keep_probabilty")  # Dropout probability
    image = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input_image")  # Input image batch first dimension image number second dimension width third dimension height 4 dimension RGB

    # Build net and load intial weights (weights before training)
    Net = BuildNetVgg16.BUILD_NET_VGG16(vgg16_npy_path=model_path)  # Create class instance for the net
    Net.build(image, NUM_CLASSES, keep_prob)  
    
    # Data reader for validation/testing images
    ValidReader = Data_Reader.Data_Reader(Image_Dir,  BatchSize=1)
    
    sess = tf.Session() #Start Tensorflow session

    print("Setting up Saver...")
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state(logs_dir)
    # if train model exist restore it
    if ckpt and ckpt.model_checkpoint_path: 
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")
    else:
        print("ERROR NO TRAINED MODEL IN: "+ckpt.model_checkpoint_path+" See Train.py for creating train network ")
        sys.exit()

    # Create output directories for predicted label
    if not os.path.exists(Pred_Dir): os.makedirs(Pred_Dir)
    if not os.path.exists(Pred_Dir+"/OverLay"): os.makedirs(Pred_Dir+"/OverLay")
    if not os.path.exists(Pred_Dir + "/Label"): os.makedirs(Pred_Dir + "/Label")

    
    print("Running Predictions:")
    print("Saving output to:" + Pred_Dir)
    # Go over all images and predict semantic segmentation in various classes
    fim = 0
    print("Start Predicting " + str(ValidReader.NumFiles) + " images")
    while (ValidReader.itr < ValidReader.NumFiles):
        fim += 1      
        print(str(fim * 100.0 / ValidReader.NumFiles) + "%")

        # Load image
        FileName=ValidReader.OrderedFiles[ValidReader.itr] # Get input image name
        Images = ValidReader.ReadNextBatchClean() # Load testing image

        # Predict annotation using net
        LabelPred = sess.run(Net.Pred, feed_dict={image: Images, keep_prob: 1.0})
        
        # Save predicted labels overlay on images
        misc.imsave(Pred_Dir + "/OverLay/"+ FileName+NameEnd  , Overlay.OverLayLabelOnImage(Images[0],LabelPred[0], w)) #Overlay label on image
        misc.imsave(Pred_Dir + "/Label/" + FileName[:-4] + ".png" + NameEnd, LabelPred[0].astype(np.uint8))

In [ ]:
##############
# Run Script #
##############

evaluatingFCN()
print("Finished")